# g4.py — Raman Map Visualization Notebook

Documentation for `Raman Maps and Hebex Graphene/g4.py`, focusing on input data, processing steps, formulas, and the figures generated.

## Purpose

- Load Raman maps exported as TSV with WITec-like MultiIndex headers.
- Smooth spectra, evaluate band-specific signal-to-noise ratios (SNR), and compute band maxima.
- Generate intensity ratio maps (I(2D)/I(G) and I(D)/I(G)) along with band intensity maps.
- Provide an interactive spectrum viewer panel and clipboard-copy functionality per subplot.

## Input data

- TSV file with two-level header (`pandas.MultiIndex`).
- Column 0: wavenumbers in cm⁻¹.
- Remaining columns: intensities per pixel, labelled `Intensity (m/n)` where `m` and `n` are integer pixel indices.
- Script reconstructs a cube `intensities[row, col, k]` and flips Y (`[::-1, :, :]`) so the origin matches the map orientation.
- Command-line arguments (also passed by the GUI `Maps-Graphene.py`):
  - `filename`: TSV path.
  - `area_size_x`, `area_size_y`: map dimensions in µm (used for `extent` in plots).

## Smoothing and signal-to-noise ratio



- Each spectrum is smoothed with a Savitzky–Golay filter (`window_length = 7`, polynomial order 3).

- For band $B$, define the in-band index set $\mathcal{I}_B$ and noise indices $\mathcal{N}_B$.

- Peak and noise metrics:

  - $\text{peak}_B = \max_{i\in\mathcal{I}_B} s_i$

  - Mean/standard deviation over the noise windows: $\mu_{\text{noise}}, \sigma_{\text{noise}}$

- SNR criterion (threshold = 3):



  $$\mathrm{SNR}_B = \frac{\text{peak}_B - \mu_{\text{noise}}}{\sigma_{\text{noise}}}. $$



- Pixels with $\mathrm{SNR}_B < 3$ are rejected for that band.


## Band maxima and ratios



For each pixel $(i, j)$ that passes the SNR check:

- Band maxima: $I_D = \max_{x \in \text{D band}} s(x)$, likewise for $I_G$ and $I_{2D}$.

- Ratios (with $\varepsilon = 10^{-5}$ to avoid division by zero):



  $$R_{2D/G} = \frac{I_{2D}}{I_G + \varepsilon}, \qquad R_{D/G} = \frac{I_D}{I_G + \varepsilon}. $$



- Pixels failing the ratio prerequisites are set to 0 and subsequently masked to white (`np.nan`) in the plots.


## Band maxima and ratios



For each pixel $(i, j)$ that passes the SNR check:

- Band maxima: $I_D = \max_{x \in \text{D band}} s(x)$, likewise for $I_G$ and $I_{2D}$.

- Ratios (with $\varepsilon = 10^{-5}$ to avoid division by zero):



  $$R_{2D/G} = \frac{I_{2D}}{I_G + \varepsilon}, \qquad R_{D/G} = \frac{I_D}{I_G + \varepsilon}. $$



- Pixels failing the ratio prerequisites are set to 0 and subsequently masked to white (`np.nan`) in the plots.


## Figure layout and colormaps

The script produces a single figure with six subplots (2 × 3).

1. **D band intensity** (top-left)
   - Colormap: Reds with `NaN → white`.
   - Color scale shared across D, G, 2D maps; global `vmax = 1.25 × max(mean intensity per band)`.
2. **G band intensity** (top-middle)
   - Colormap: Greens, same scale.
3. **2D band intensity** (top-right)
   - Colormap: Blues, same scale.
4. **I(2D)/I(G) ratio** (bottom-left)
   - Custom gradient (0 → white, 0.01–0.5 → reds, 0.5–1 → orange, 1–1.6 → yellow, 1.6–2.5 → light green, 2.5–3 → green, 3–3.5 → dark green).
   - Range: 0 to 3.5, ticks every 0.5.
5. **I(D)/I(G) ratio** (bottom-middle)
   - White → pink → red → dark red.
   - Range: 0 to 1.
6. **Spectrum viewer** (bottom-right)
   - Clicking any map displays the corresponding smoothed spectrum.
   - Right-click (or context menu) copies the selected panel image to the clipboard, using Qt or Windows clipboard backends if available.

## Libraries

- `numpy`, `pandas`, `matplotlib`
- `scipy.signal.savgol_filter`
- `PIL.Image`, optional `pywin32` and `PyQt5`/`PySide2` for clipboard support
- `tkinter` (indirectly, when launching via `Maps-Graphene.py`)

## Interaction summary

- **Left-click** on D/G/2D/ratio maps → updates the spectrum panel with the selected pixel.
- **Right-click** → copies the axes image to clipboard (if backend supports it).

## Notes

- Maps are flipped vertically to match expected orientation.
- Masked pixels (SNR failures or zero intensities) appear white in the visualizations.
- Adjust band windows, noise ranges, or ratio limits if working with different acquisitions.
- For batch processing or saving each panel separately, modify the section near the end of `g4.py`.